### 1. إعداد المكتبات والمتغيرات
تحديد المسارات (Data, DB) وتحميل المكتبات اللازمة مثل `chromadb` و `sentence_transformers`.

In [1]:
# الخلية 1: الاستيرادات والإعدادات
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import os
import shutil 

# ==========================================
# إعدادات المشروع
# ==========================================
INPUT_CSV = 'medical_data_cleaned.csv'
DB_PATH = "./chroma_db_storage"
COLLECTION_NAME = "medical_knowledge_base"
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

print("✅ تم إعداد المسارات بنجاح.")

/home/ahmed/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ تم إعداد المسارات بنجاح.


### 2. تحميل النموذج والبيانات
تحميل نموذج التضمين (Embedding Model) وقراءة ملف البيانات المعالج سابقاً للتأكد من جاهزيته.

In [2]:
# الخلية 2: تحميل الموديل وقراءة ملف الـ CSV
try:
    print(f"⏳ جاري تحميل نموذج الذكاء الاصطناعي: {MODEL_NAME}...")
    embedding_model = SentenceTransformer(MODEL_NAME)
    print("✅ تم تحميل النموذج بنجاح.")

    print(f"⏳ جاري قراءة الملف: {INPUT_CSV}...")
    if not os.path.exists(INPUT_CSV):
        # Fallback for robustness
        print(f"⚠️ الملف {INPUT_CSV} غير موجود، جارٍ البحث في dataset/...")
        if os.path.exists('dataset/medical_data_cleaned.csv'):
            INPUT_CSV = 'dataset/medical_data_cleaned.csv'
    
    df = pd.read_csv(INPUT_CSV, encoding='utf-8-sig')
    
    required_cols = ['cleaned_question', 'cleaned_answer']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"❌ الأعمدة المطلوبة غير موجودة: {required_cols}")
        
    print(f"✅ تم تحميل البيانات: {len(df)} سجل.")
    
except Exception as e:
    print(f"❌ حدث خطأ: {e}")

⏳ جاري تحميل نموذج الذكاء الاصطناعي: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...
✅ تم تحميل النموذج بنجاح.
⏳ جاري قراءة الملف: medical_data_cleaned.csv...
✅ تم تحميل البيانات: 808471 سجل.


### 3. تهيئة قاعدة البيانات
إنشاء اتصال مع ChromaDB (Persistent Client) وإعداد المجموعة (Collection) لتخزين المتجهات.

In [3]:
# الخلية 3: تهيئة قاعدة البيانات
# تنبيه: هذا الكود يقوم بإنشاء قاعدة جديدة، إذا كانت موجودة سيقوم بحذفها وإعادة إنشائها لضمان التحديث

print("⚙️ جاري إعداد قاعدة البيانات...")

# استخدام PersistentClient للحفظ الدائم على القرص
client = chromadb.PersistentClient(path=DB_PATH)

# التحقق مما إذا كانت المجموعة موجودة وحذفها (لبدء نظيف)
try:
    client.delete_collection(name=COLLECTION_NAME)
    print(f"   - تم حذف المجموعة القديمة '{COLLECTION_NAME}' لتحديث البيانات.")
except:
    print(f"   - جاري إنشاء مجموعة جديدة '{COLLECTION_NAME}'...")

# إنشاء المجموعة (المكان الذي نخزن فيه البيانات)
collection = client.create_collection(name=COLLECTION_NAME)
print("✅ قاعدة البيانات جاهزة لاستقبال البيانات.")

⚙️ جاري إعداد قاعدة البيانات...
   - جاري إنشاء مجموعة جديدة 'medical_knowledge_base'...
✅ قاعدة البيانات جاهزة لاستقبال البيانات.


### 4. الفهرسة والتخزين
تقسيم البيانات إلى دفعات (Batches)، تحويل النصوص إلى متجهات (Embeddings)، وتخزينها مع البيانات الوصفية في قاعدة البيانات.

In [4]:
# الخلية 4: التضمين والتخزين (محسنة)
print("🚀 بدء عملية الفهرسة (Indexing)...")

BATCH_SIZE = 64 
total_rows = len(df)
print_interval = 500 # تقليل عدد مرات الطباعة لتسريع الواجهة

for i in range(0, total_rows, BATCH_SIZE):
    end_i = min(i + BATCH_SIZE, total_rows)
    batch_df = df.iloc[i:end_i]
    
    batch_ids = [str(idx) for idx in batch_df.index]
    batch_docs = batch_df['cleaned_question'].astype(str).tolist()
    
    # Generate embeddings
    batch_embeddings = embedding_model.encode(batch_docs).tolist()
    
    # Optimization: Vectorized metadata creation
    # Ensure columns exist and fillna
    meta_df = batch_df.copy()
    if 'Category' not in meta_df.columns:
        meta_df['Category'] = 'General'
    
    meta_df = meta_df.rename(columns={
        'cleaned_answer': 'answer',
        'Question': 'original_question',
        'Category': 'category'
    })
    
    # Select only relevant columns for metadata to avoid errors
    batch_metadatas = meta_df[['answer', 'original_question', 'category']].to_dict('records')
    
    collection.add(
        ids=batch_ids,
        documents=batch_docs,
        embeddings=batch_embeddings,
        metadatas=batch_metadatas
    )
    
    if i % (BATCH_SIZE * 10) == 0: # Print every 10 batches roughly
        print(f"   - Progress: {i}/{total_rows} ({round(i/total_rows*100, 1)}%)")

print("-" * 50)
print(f"🎉 تم الانتهاء! تم حفظ قاعدة البيانات في: {DB_PATH}")